# FUNCIONA

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

!pip install transformers[torch]
!pip install evaluate
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_metric, Dataset
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/FULL-Seminario/Proyectos/rac_gpt/prompts/pruebas/PREGUNTAS COMPLETAS/RAC1_copia/Copia de PR2_35_G.txt'

# Leer el contenido del archivo
with open(file_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

In [ ]:
# Dividir el contenido en preguntas y respuestas
lines = file_content.split('\n')
data = []
for i in range(0, len(lines)-1, 2):
    question = lines[i].strip()
    answer = lines[i+1].strip()
    data.append({"question": question, "answer": answer})

In [ ]:
# Crear el conjunto de datos
dataset = Dataset.from_dict({"question": [item["question"] for item in data], "answer": [item["answer"] for item in data]})

In [ ]:
# Agregar etiquetas al conjunto de datos
# Asumamos que todas las entradas tienen la misma etiqueta, por ejemplo, "0"
dataset = dataset.add_column("label", [0] * len(dataset))

In [ ]:
# Cargar el modelo y el tokenizador
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Definir la función de tokenización
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["answer"], padding="max_length", truncation=True)

In [ ]:
# Tokenizar el conjunto de datos
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

In [ ]:
# Inicializar el modelo de clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Definir la métrica
metric_name = "accuracy"
metric = load_metric(metric_name)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=1)
    labels = p.label_ids

    # Métricas básicas
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)

    # Informe de clasificación para obtener la métrica 'support'
    classification_rep = classification_report(labels, predictions, output_dict=True)
    support = classification_rep['weighted avg']['support']

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "support": support}

Accuracy (Exactitud):
Fórmula: (Verdaderos Positivos + Verdaderos Negativos) / Total
La exactitud mide la proporción de predicciones correctas en relación con el número total de predicciones. Es una medida general de qué tan bien el modelo clasifica las instancias.

Precision (Precisión):
Fórmula: Verdaderos Positivos / (Verdaderos Positivos + Falsos Positivos)
La precisión se centra en las instancias clasificadas como positivas por el modelo y mide cuántas de ellas realmente son positivas. Es útil cuando el coste de los falsos positivos es alto.

Recall (Recuperación o Sensibilidad):
Fórmula: Verdaderos Positivos / (Verdaderos Positivos + Falsos Negativos)
El recall evalúa la capacidad del modelo para capturar todas las instancias positivas. Es útil cuando el coste de los falsos negativos es alto.

F1 Score (Puntuación F1):
Fórmula: 2 * (Precision * Recall) / (Precision + Recall)
La puntuación F1 es la media armónica de precisión y recall. Proporciona un equilibrio entre la precisión y el recall. Es útil cuando hay un desequilibrio entre las clases y se busca un rendimiento equitativo en ambas.

In [ ]:
# Configurar el entrenamiento
training_args = TrainingArguments(
    output_dir="./sentiment_analysis",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

# Configurar el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics,
)


In [ ]:
# Entrenar el modelo
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=153, training_loss=0.024039831036835713, metrics={'train_runtime': 118.6452, 'train_samples_per_second': 10.139, 'train_steps_per_second': 1.29, 'total_flos': 316531125384192.0, 'train_loss': 0.024039831036835713, 'epoch': 3.0})

In [ ]:
# Evaluar el modelo
eval_results = trainer.evaluate()

# Imprimir resultados de evaluación
print("Resultados de Evaluación:", eval_results)


Resultados de Evaluación: {'eval_loss': 0.000464211538201198, 'eval_accuracy': 1.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_support': 401, 'eval_runtime': 13.8588, 'eval_samples_per_second': 28.935, 'eval_steps_per_second': 3.68, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
# Obtener las predicciones en el conjunto de datos de evaluación
predictions = trainer.predict(tokenized_dataset)

# Acceder a las predicciones
predicted_labels = predictions.predictions.argmax(axis=1)

# Comparar con las etiquetas reales
true_labels = tokenized_dataset["label"]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
